In [1]:
import xml.etree.ElementTree as ET 
import gzip
from xml.etree.cElementTree import ElementTree
import statistics

In [2]:
def population_stats(population):
    number=len(population)
    print("Taille population :   ",number)
    driving_lic_poss = len([person for person in population.iter('person') if person[0][6].text=="yes"])
    print("Titulaires du permis :",driving_lic_poss," soit ", driving_lic_poss/number * 100, "%")
    women = len([person for person in population.iter('person') if person[0][13].text=="f"])
    print("Femmes :              ", women, " soit ", women/number * 100, "%")
    men = len([person for person in population.iter('person') if person[0][13].text=="m"])
    print("Hommes :              ", men, " soit ", men/number * 100, "%")
    print("Revenu moyen :        ",statistics.mean([float(person[0][9].text) for person in population.iter('person')]))
    pt_sub = len([person for person in population.iter('person') if person[0][7].text=="true"])
    print("Abonnés aux pt :  ", pt_sub, " soit ", pt_sub/number * 100, "%")
    #return members
    #CSMembership(members,companies,CSMembership_file

## Loading data

In [3]:
##Input plan
initialPlanFile_v1 = '/media/aodiallo/Data/MATSIM/output/output_grand_lyon_v0.25/lyon_metropole_population_v1.xml.gz'
treePopulation = ET.parse(gzip.open(initialPlanFile_v1,'r'))
population  = treePopulation.getroot()

## Some population statistics

In [4]:
population_stats(population)

Taille population :    315786
Titulaires du permis : 197306  soit  62.48092062346019 %
Femmes :               164966  soit  52.23980797122102 %
Hommes :               150819  soit  47.759875358628946 %
Revenu moyen :         3611.535479282468
Abonnés aux pt :   107570  soit  34.06420803962177 %


In [5]:
"""
for person in population.iter('person'):
    if person[0][6].text=="yes":
        print("yesssssssssssssssssss")
"""

'\nfor person in population.iter(\'person\'):\n    if person[0][6].text=="yes":\n        print("yesssssssssssssssssss")\n'

In [6]:
print("Nombre d'agents = ", len(population) -1)#Without the CRS attribute!!!

Nombre d'agents =  315785


In [7]:
population.tag

'population'

In [8]:
population.attrib

{}

In [9]:
population[0].tag, population[0].attrib

('attributes', {})

In [10]:
population[0][0].text

'Atlantis'

In [11]:
for person in population.iter('person'):
    print(person.tag, person.attrib)
    print("number of element ", len(person))
    print(person[0][0].tag, person[0][0].attrib)
    print(person[0][1].tag, person[0][1].attrib)
    print(person[0][2].tag, person[0][2].attrib)
    
    print("number of element of attribute ", len(person[0]))
    
    for i in range(len(person[0])):
        print(person[0][i].tag, person[0][i].attrib)
    
    print(person[1][0].tag, person[1][0].attrib)
    
    print("number of element of plan ", len(person[1]))
    
    for i in range(len(person[1])):
        if person[1][i].get('type')=='home':
            print("The activity is home !!!")
            print("And his link id is ", person[1][i].get('link'))
            break
        else :
            print("There is no HOME activity !")
    break

person {'id': '10000'}
number of element  2
attribute {'name': 'age', 'class': 'java.lang.Integer'}
attribute {'name': 'bikeAvailability', 'class': 'java.lang.String'}
attribute {'name': 'carAvailability', 'class': 'java.lang.String'}
number of element of attribute  15
attribute {'name': 'age', 'class': 'java.lang.Integer'}
attribute {'name': 'bikeAvailability', 'class': 'java.lang.String'}
attribute {'name': 'carAvailability', 'class': 'java.lang.String'}
attribute {'name': 'censusHouseholdId', 'class': 'java.lang.Long'}
attribute {'name': 'censusPersonId', 'class': 'java.lang.Long'}
attribute {'name': 'employed', 'class': 'java.lang.String'}
attribute {'name': 'hasLicense', 'class': 'java.lang.String'}
attribute {'name': 'hasPtSubscription', 'class': 'java.lang.Boolean'}
attribute {'name': 'householdId', 'class': 'java.lang.Integer'}
attribute {'name': 'householdIncome', 'class': 'java.lang.Double'}
attribute {'name': 'htsHouseholdId', 'class': 'java.lang.Long'}
attribute {'name': 'h

In [12]:
householdIdList=[]
motorHousehold=[]
linkHousehold=[]
cpt = 0
cpt_persons = 0

# Regarder si le ménage est motorisé  pour relever son householdId et linkHousehold
for person in population.iter('person'):
    cpt = cpt +1
    carAvail=person[0][2]#car availability attribute. Possible values : some, all, none. The value never does not exist
    householdId=person[0][8]## cahnge person[0][5] to person[0][8]
    
    ##Determine the number of household
    if householdId.text not in householdIdList :
        householdIdList.append(householdId.text)
    
    noHomeActivity = False###Its was appear that some agents (persons) has not home activity. 
    #So we will assign a PAV even they have car!
    linkHome = ""
    for i in range(len(person[1])):
        if person[1][i].get('type')=='home':
            #print("The activity is home !!!")
            linkHome = person[1][i].get('link')
            noHomeActivity = False
            break
        else :
            noHomeActivity = True
            #print("There is no HOME activity !")
            
    """
    ##Search the home activity
    for i in range(len(person[1])):
        if person[1][i].get('type')=='home':
            linkHome=person[1][i].get('link')
            break
        else :
            print("There is no HOME activity !")
        
    """
    if carAvail.text!='none' and not noHomeActivity :
        cpt_persons = cpt_persons +1
        householdAVId=person[0][8]## cahnge person[0][5] to person[0][8]
        if householdAVId.text not in motorHousehold:##Note: A household is composed of one or more persons 
            motorHousehold.append(householdAVId.text)
            linkHousehold.append(linkHome)

In [13]:
print('Number of household with a PAV: ', len(motorHousehold), ' on ', len(householdIdList), " households total corresponding to ", cpt, " persons!")
print('Or : ', len(motorHousehold)/len(householdIdList) * 100, " % of total households")

Number of household with a PAV:  102240  on  153867  households total corresponding to  315785  persons!
Or :  66.44699643198346  % of total households


In [14]:
cpt_persons

241095

In [15]:
#20512/30900 *100
#20510/30900 *100

66.37540453074435

In [16]:
#Modifier les plans pour retirer la voiture et imposer les modes private_AV aux utilisateurs car et car_passenger
for person in population.iter('person'):
    carAvail=person[0][2]
    householdId=person[0][8]## cahnge person[0][5] to person[0][8]
    if householdId.text in motorHousehold: #si motorisé 
        for leg in person[1].iter('leg'):
            if leg.get('mode') == 'car':
                leg.set('mode','private_AV')
                leg[0][0].text='private_AV'## <atributes><attribute>value</attribute></attributes>
                if leg[1].get('start_link') == leg[1].get('end_link'): #éviter les erreurs car une requête start_link=end_link échoue
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
            if leg.get('mode') == 'car_passenger':
                leg.set('mode','private_AV')
                leg[0][0].text='private_AV'
                if leg[1].get('start_link') == leg[1].get('end_link'):
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
        subpop=ET.SubElement(person[0],'attribute', attrib={'name': "subpopulation", 'class':"java.lang.String"})
        subpop.text="private"
        carAvail.text='none'
    else:        
        subpop=ET.SubElement(person[0],'attribute', attrib={'name': "subpopulation", 'class':"java.lang.String"})
        subpop.text="no_private"
        #Not needing to disable the car availability ?
        for leg in person[1].iter('leg'):
            if leg.get('mode') == 'car':##Delete the car mode for this sub-population!
                leg.set('mode','walk')
                leg[0][0].text='walk'
                leg[1].text=''
            if leg.get('mode') == 'car_passenger':
                leg.set('mode','walk')
                leg[0][0].text='walk'
                leg[1].text=''

In [17]:
modifiedPlanFile = '/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/scenario1/population_sce1_25pcent.xml.gz'
f = gzip.open(modifiedPlanFile, 'wb')
f.write('<?xml version="1.0" encoding="utf-8"?><!DOCTYPE population SYSTEM "http://www.matsim.org/files/dtd/population_v6.dtd">'.encode('utf8'))
ElementTree(population).write(f) 
f.close()

In [18]:
stop

NameError: name 'stop' is not defined

In [ ]:
# Utiliser CreateTaxiVehicles.java pour créer le fichier de véhicules privés selon len(motorHousehold)
#  en modifiant: 
#    int numberofVehicles = 102240;
#    String networkfile = "/media/aodiallo/Data/MATSIM/output/output_grand_lyon_v0.05/lyon_metropole_network.xml.gz";
#    String taxisFile = "/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/taxis/private_vehicles_sce1_" + numberofVehicles + ".xml";
       
treeVehicles = ET.parse('/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/taxis/private_vehicles_sce1_102240.xml')
rootVehicles = treeVehicles.getroot()

In [ ]:
print(len(rootVehicles))

In [ ]:
## Redefine the ID and the START_LINK to assign taxi to correspond to the household for his PAV
i=0
for taxi in rootVehicles.iter('vehicle'):
    taxi.set('id','taxi_' + str(motorHousehold[i]))
    taxi.set('start_link',linkHousehold[i])
    i+=1

In [ ]:
g = gzip.open('/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/scenario1/private_vehicles_sce1_25pcent_20510.xml.gz', 'wb')
g.write('<?xml version="1.0" encoding="utf-8"?><!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">'.encode('utf8'))
ElementTree(rootVehicles).write(g,'utf-8')
g.close()